## Load raw data from MySQL server

In [ ]:
!pip install pymysql > /dev/null

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import json
with open('/content/drive/MyDrive/최종프로젝트/자료/mysql.json', 'r') as fp:
  config_str = fp.read()
config = json.loads(config_str)

In [ ]:
import pymysql
conn = pymysql.connect(
    host = config['host'],
    user = config['user'],
    password = config['password'],
    database = config['database'],
    port = config['port']
)

In [ ]:
import pandas as pd

sql_state = 'select * from safety1;'

df = pd.read_sql_query(sql_state, conn)
df

,year,month,day,hour,minute,second,Person,Vest,Helmet,NoHelmet,NoVest,reg_date
0,2021,11,30,15,30,38,0,0,0,0,0,2021-11-30 15:30:38
1,2021,11,30,15,30,46,0,0,0,0,0,2021-11-30 15:30:46
2,2021,11,30,15,30,54,0,0,0,0,0,2021-11-30 15:30:55
3,2021,11,30,15,31,3,0,0,0,0,0,2021-11-30 15:31:03
4,2021,11,30,15,31,10,0,0,0,0,0,2021-11-30 15:31:11
...,...,...,...,...,...,...,...,...,...,...,...,...
672,2021,12,1,13,37,50,1,1,1,0,0,2021-12-01 13:37:50
673,2021,12,1,13,37,58,0,0,0,0,0,2021-12-01 13:37:58
674,2021,12,1,13,38,7,0,0,0,0,0,2021-12-01 13:38:07
675,2021,12,1,13,38,15,0,0,0,0,0,2021-12-01 13:38:15


In [ ]:
import time
import datetime as dt
from datetime import datetime
from pytz import timezone, utc
today = dt.datetime.now(timezone('Asia/Seoul'))
today

datetime.datetime(2021, 12, 1, 11, 36, 32, 6995, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

In [ ]:
today.year, today.month, today.day, today.hour

(2021, 12, 1, 11)

In [ ]:
df.columns

Index(['year', 'month', 'day', 'hour', 'minute', 'second', 'Person', 'Vest',
       'Helmet', 'NoHelmet', 'NoVest', 'reg_date'],
      dtype='object')

In [ ]:
df.dtypes

year                object
month               object
day                 object
hour                object
minute              object
second              object
Person              object
Vest                object
Helmet              object
NoHelmet            object
NoVest              object
reg_date    datetime64[ns]
dtype: object

In [ ]:
df[['Person', 'Vest', 'Helmet', 'NoHelmet', 'NoVest']] = df[['Person', 'Vest', 'Helmet', 'NoHelmet', 'NoVest']].apply(pd.to_numeric)
df

,year,month,day,hour,minute,second,Person,Vest,Helmet,NoHelmet,NoVest,reg_date
0,2021,11,30,15,30,38,0,0,0,0,0,2021-11-30 15:30:38
1,2021,11,30,15,30,46,0,0,0,0,0,2021-11-30 15:30:46
2,2021,11,30,15,30,54,0,0,0,0,0,2021-11-30 15:30:55
3,2021,11,30,15,31,3,0,0,0,0,0,2021-11-30 15:31:03
4,2021,11,30,15,31,10,0,0,0,0,0,2021-11-30 15:31:11
...,...,...,...,...,...,...,...,...,...,...,...,...
672,2021,12,1,13,37,50,1,1,1,0,0,2021-12-01 13:37:50
673,2021,12,1,13,37,58,0,0,0,0,0,2021-12-01 13:37:58
674,2021,12,1,13,38,7,0,0,0,0,0,2021-12-01 13:38:07
675,2021,12,1,13,38,15,0,0,0,0,0,2021-12-01 13:38:15


In [ ]:
df.dtypes

year                object
month               object
day                 object
hour                object
minute              object
second              object
Person               int64
Vest                 int64
Helmet               int64
NoHelmet             int64
NoVest               int64
reg_date    datetime64[ns]
dtype: object

In [ ]:
df_groupby = df.groupby(['year', 'month', 'day', 'hour']).sum()
df_groupby.reset_index(inplace=True)
df_groupby

,year,month,day,hour,Person,Vest,Helmet,NoHelmet,NoVest
0,2021,11,30,15,0,0,0,0,0
1,2021,11,30,16,52,21,53,0,5
2,2021,11,30,17,9,3,21,0,0
3,2021,12,1,10,34,14,34,0,0
4,2021,12,1,11,12,9,10,0,0
5,2021,12,1,12,9,1,16,0,1
6,2021,12,1,13,5,2,4,0,0
7,2021,12,1,9,15,3,23,0,2


In [ ]:
with open('result_hour_sum.html', 'w') as fo:
    fo.write(df_groupby.to_html())

In [ ]:
with open('result_hour_sum.csv', 'w') as fo:
    fo.write(df_groupby.to_csv())

In [ ]:
# 헬멧 + 노헬멧 >= 사람 인 경우 사람=헬멧+노헬멧, 조끼 + 노조끼 >= 사람 인 경우 사람=조끼+노조끼
df_groupby_new = df_groupby.copy()
df_groupby_new['Person'] = df_groupby_new.apply(lambda row: row['Helmet'] + row['NoHelmet'] if row['Helmet'] + row['NoHelmet'] >= row['Person'] else row['Person'],axis=1)
df_groupby_new['Person'] = df_groupby_new.apply(lambda row: row['Vest'] + row['NoVest'] if row['Vest'] + row['NoVest'] >= row['Person'] else row['Person'],axis=1) 
df_groupby_new

,year,month,day,hour,Person,Vest,Helmet,NoHelmet,NoVest
0,2021,11,30,15,0,0,0,0,0
1,2021,11,30,16,53,21,53,0,5
2,2021,11,30,17,21,3,21,0,0
3,2021,12,1,10,34,14,34,0,0
4,2021,12,1,11,12,9,10,0,0
5,2021,12,1,12,16,1,16,0,1
6,2021,12,1,13,5,2,4,0,0
7,2021,12,1,9,23,3,23,0,2


In [ ]:
def cal_hel(row):
  if row.Helmet == row.Person == 0 :               # 1) 헬멧 객체와 사람객체가 0인 경우 미표기
    return '-'
  elif row.Helmet >= row.Person :                  # 2) 헬멧 객체가 사람보다 많은경우 100% 착용
    return '100%'
  elif row.Helmet == row.Person :                  # 3) 헬멧 객체와 사람객체가 같은경우 100% 착용
    return '100%'
  elif row.Helmet < row.Person :                   # 4) 사람 > 헬멧 인 경우 헬멧/사람*100 으로 착용률 계산
    return f'{row.Helmet / row.Person * 100:.1f}%'
  elif row.NoHelmet == 0:                          # 5) 노헬멧이 0이면  미표기
    return '-'
  else :                                           # 6) 그외  미표기
    return '-'


def cal_vest(row):
  if row.Vest == row.Person == 0 :                # 1) 조끼 객체와 사람객체가 0인 경우 미표기
    return '-'
  elif row.Vest >= row.Person :                   # 2) 조끼 객체가 사람보다 많은경우 100% 착용
    return '100%'
  elif row.Vest == row.Person :                   # 3) 조끼 객체와 사람객체가 같은경우 100% 착용
    return '100%'
  elif row.Vest < row.Person :                    # 4) 사람 > 조끼 인 경우 조끼/사람*100 으로 착용률 계산
    return f'{row.Vest / row.Person * 100:.1f}%'
  elif row.NoVest == 0:                           # 5) 노조끼가 0이면  미표기
    return '-'
  else :                                          # 6) 그외  미표기
    return '-'

In [ ]:
df_groupby_hour = df_groupby_new[['year', 'month', 'day', 'hour']].copy()
df_groupby_hour.reset_index(inplace=True)
df_groupby_hour

,index,year,month,day,hour
0,0,2021,11,30,15
1,1,2021,11,30,16
2,2,2021,11,30,17
3,3,2021,12,1,10
4,4,2021,12,1,11
5,5,2021,12,1,12
6,6,2021,12,1,13
7,7,2021,12,1,9


In [ ]:
df_groupby_hour = df_groupby_new[['year', 'month', 'day', 'hour']].copy()
df_groupby_hour['헬멧착용률'] = df_groupby_new.apply(cal_hel, axis=1)
df_groupby_hour['조끼착용률'] = df_groupby_new.apply(cal_vest, axis=1)
df_groupby_hour.rename(columns = {'year':'연도', 'month':'월',	'day':'일',	'hour':'시간'}, inplace=True)
df_groupby_hour

,연도,월,일,시간,헬멧착용률,조끼착용률
0,2021,11,30,15,-,-
1,2021,11,30,16,100%,39.6%
2,2021,11,30,17,100%,14.3%
3,2021,12,1,10,100%,41.2%
4,2021,12,1,11,83.3%,75.0%
5,2021,12,1,12,100%,6.2%
6,2021,12,1,13,80.0%,40.0%
7,2021,12,1,9,100%,13.0%


In [ ]:
df_groupby_day = df_groupby_new[['year', 'month', 'day']].copy()
df_groupby_day['헬멧착용률'] = df_groupby_new.apply(cal_hel, axis=1)
df_groupby_day['조끼착용률'] = df_groupby_new.apply(cal_vest, axis=1)
df_groupby_day.rename(columns = {'year':'연도',	'month':'월',	'day':'일'}, inplace=True)
df_groupby_day

,연도,월,일,헬멧착용률,조끼착용률
0,2021,11,30,-,-
1,2021,11,30,100%,39.6%
2,2021,11,30,100%,14.3%
3,2021,12,1,100%,41.2%
4,2021,12,1,83.3%,75.0%
5,2021,12,1,100%,6.2%
6,2021,12,1,80.0%,40.0%
7,2021,12,1,100%,13.0%


In [ ]:
df_groupby_month = df_groupby_new[['year', 'month']].copy()
df_groupby_month['헬멧착용률'] = df_groupby_new.apply(cal_hel, axis=1)
df_groupby_month['조끼착용률'] = df_groupby_new.apply(cal_vest, axis=1)
df_groupby_month.rename(columns = {'year':'연도',	'month':'월'}, inplace=True)
df_groupby_month

,연도,월,헬멧착용률,조끼착용률
0,2021,11,-,-
1,2021,11,100%,39.6%
2,2021,11,100%,14.3%
3,2021,12,100%,41.2%
4,2021,12,83.3%,75.0%
5,2021,12,100%,6.2%
6,2021,12,80.0%,40.0%
7,2021,12,100%,13.0%


In [ ]:
df_groupby_year = df_groupby_new[['year']].copy()
df_groupby_year['헬멧착용률'] = df_groupby_year.apply(cal_hel, axis=1)
df_groupby_year['조끼착용률'] = df_groupby_year.apply(cal_vest, axis=1)
df_groupby_year.rename(columns = {'year':'연도'}, inplace=True)
df_groupby_year

,연도,헬멧착용률,조끼착용률
0,2021,100%,39.0%


In [ ]:
df_groupby_new.to_html('df_groupby_new.html')

In [ ]:
df_groupby_hour.to_csv('df_groupby_hour.csv')
df_groupby_day.to_csv('df_groupby_day.csv')
df_groupby_month.to_csv('df_groupby_month.csv')
df_groupby_year.to_csv('df_groupby_year.csv')

## web view

In [ ]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()

In [ ]:
# 스캐터 relation
p = figure(plot_width=700, plot_height=400)

In [ ]:
import jinja2
from bokeh.embed import components
template = jinja2.Template('''\
<html>
<link href='http://cdn.pydata.org/bokeh/release/bokeh-1.0.2.min.css' rel='stylesheet' type='text/css'>
<script src='http://cdn.pydata.org/bokeh/release/bokeh-1.0.2.min.js'></script>
<head><title>bokeh test</title>
<head>
<body>
<h1> 건설현장 안전장비 착용 관리 </h1>
<br>
<br>
안전장비 착용률 통계(시간단위)
<br>
<br>


<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>year</th>
      <th>month</th>
      <th>day</th>
      <th>hour</th>
      <th>wear_helmet</th>
      <th>wear_vest</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2021</td>
      <td>11</td>
      <td>30</td>
      <td>15</td>
      <td>-</td>
      <td>-</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2021</td>
      <td>11</td>
      <td>30</td>
      <td>16</td>
      <td>100%</td>
      <td>90.4%</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2021</td>
      <td>11</td>
      <td>30</td>
      <td>17</td>
      <td>100%</td>
      <td>-</td>
    </tr>
  </tbody>
</table>


{{script}}
{{div}}

</body>
</html>
'''
)

In [ ]:
script, div = components(p)

from IPython.display import HTML
HTML(template.render(script=script, div=div))

,year,month,day,hour,wear_helmet,wear_vest
0,2021,11,30,15,-,-
1,2021,11,30,16,100%,90.4%
2,2021,11,30,17,100%,-


In [ ]:
with open('a.html', 'w') as f:
  f.write(template.render(script=script, div=div))

import webbrowser
webbrowser.open('a.html')

False